In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
os.environ["WANDB_DISABLED"] = "true"
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tosdr-terms-of-service-corpus/text/DigitalTurbine_CookiePolicy.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/Juicer_CookiePolicy.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/f95zone_Cookie.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/EasyChair_TermsofService.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/AnimePlanet_TermsofUse.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/NabuCasa_PrivacyPolicy.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/ByGabyArdigo_RefundPolicy.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/IPVanish_TermsofService.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/GrowTix_TermsandConditions.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/TheSession_API.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/AdGuardSoftwareLimited_EndUserLicenceAgreement.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/BulkImageDownloader_PrivacyPolicy.txt
/kaggle/input/tosdr-terms-of-service-corpus/text/WriterDu

In [11]:
import os
os.listdir("/kaggle/input/tosdr-terms-of-service-corpus")


['text']

In [12]:
import os
import pandas as pd

# Caminho base dos arquivos
base_path = "/kaggle/input/tosdr-terms-of-service-corpus/text"

# Lista de arquivos
files = os.listdir(base_path)

# Criar lista de dicionários com nome e conteúdo
data = []
for file in files:
    file_path = os.path.join(base_path, file)
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
        data.append({"arquivo": file, "texto": text})

# Criar DataFrame
df = pd.DataFrame(data)
df.head()

,arquivo,texto
0,DigitalTurbine_CookiePolicy.txt,\n App Developers Brands &amp.\nAgenci...
1,Juicer_CookiePolicy.txt,\n Cookie Policy for Juicer.ioWhat Are ...
2,f95zone_Cookie.txt,\nThis page discusses how cookies are used by ...
3,EasyChair_TermsofService.txt,\n Help/Log inEasyChair Terms of Servic...
4,AnimePlanet_TermsofUse.txt,"\n\nAnime-Planet, including its successor or a..."


In [13]:
import re

def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-ZÀ-ÿ0-9.,;:!?() ]', '', texto)
    return texto.strip()

df["texto_limpo"] = df["texto"].apply(limpar_texto)

# Gerar rótulos automáticos com base em palavras-chave
def gerar_label(texto):
    texto = texto.lower()
    # Classe 2 = "ruim" (violação de privacidade, coleta de dados, compartilhamento)
    if any(p in texto for p in ["compartilhar dados", "publicidade", "terceiros", "cookies", "armazenar informações"]):
        return 2
    # Classe 0 = "bom" (direitos do usuário, transparência, segurança)
    elif any(p in texto for p in ["direito", "privacidade", "proteger dados", "excluir conta", "transparente"]):
        return 0
    # Classe 1 = "neutro"
    else:
        return 1

df["classe"] = df["texto_limpo"].apply(gerar_label)
df["classe"].value_counts()


classe
2    5394
1    4102
Name: count, dtype: int64

In [14]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizar os textos
tokens = tokenizer(df["texto_limpo"].tolist(), padding=True, truncation=True, max_length=256, return_tensors="pt")


In [15]:
from sklearn.model_selection import train_test_split
import torch

X_train, X_test, y_train, y_test = train_test_split(tokens["input_ids"], df["classe"].values, test_size=0.2, random_state=42)


In [16]:
from torch.utils.data import Dataset

class ContratoDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": torch.tensor(self.labels[idx])
        }

train_dataset = ContratoDataset(X_train, y_train)
test_dataset = ContratoDataset(X_test, y_test)


In [17]:
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import os
import pandas as pd
import re

# Caminho base dos arquivos
base_path = "/kaggle/input/tosdr-terms-of-service-corpus/text"
files = os.listdir(base_path)

# Criar lista de dicionários com nome e conteúdo
data = []
for file in files:
    file_path = os.path.join(base_path, file)
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
        data.append({"arquivo": file, "texto": text})

# Criar DataFrame
df = pd.DataFrame(data)

# Limpar texto
def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'[^a-zA-ZÀ-ÿ0-9.,;:!?() ]', '', texto)
    return texto.strip()

df["texto_limpo"] = df["texto"].apply(limpar_texto)

# Criar labels automáticas
def gerar_label(texto):
    texto = texto.lower()
    if any(p in texto for p in ["compartilhar dados", "publicidade", "terceiros", "cookies", "armazenar informações"]):
        return 2  # ruim
    elif any(p in texto for p in ["direito", "privacidade", "proteger dados", "excluir conta", "transparente"]):
        return 0  # bom
    else:
        return 1  # neutro

df["classe"] = df["texto_limpo"].apply(gerar_label)


In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import os
os.environ["WANDB_DISABLED"] = "true"
from sklearn.model_selection import train_test_split

# 1️⃣ Definir modelo base
model_name = "distilbert-base-uncased"  # ou "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2️⃣ Tokenizar textos
tokens = tokenizer(df["texto_limpo"].tolist(), padding=True, truncation=True, max_length=256, return_tensors="pt")

# 3️⃣ Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(tokens["input_ids"], df["classe"].values, test_size=0.2, random_state=42)

# 4️⃣ Criar Dataset PyTorch
class ContratoDataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "labels": torch.tensor(self.labels[idx])
        }

train_dataset = ContratoDataset(X_train, y_train)
test_dataset = ContratoDataset(X_test, y_test)

# 5️⃣ Carregar modelo de classificação
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 6️⃣ Definir argumentos de treino
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs"
)
# 7️⃣ Criar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 8️⃣ Iniciar treino
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.506700
1000,0.475600
1500,0.470300
2000,0.395200
2500,0.409600
3000,0.417900
3500,0.408000


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


TrainOutput(global_step=3798, training_loss=0.4346447483120999, metrics={'train_runtime': 11878.1555, 'train_samples_per_second': 1.279, 'train_steps_per_second': 0.32, 'total_flos': 1006240304738304.0, 'train_loss': 0.4346447483120999, 'epoch': 2.0})